In [82]:
import networkx as nx 
import numpy as np
import json
import pandas as pd
from sage.graphs.graph_input import from_dict_of_lists
import time

In [2]:
def nakljucni_MIS(G, P):
    I = []            #Nastajajoca neodvisna mnozica
    V = list(G.nodes)
    for i in V:        # V tej mnozici izberemo vozlisca, ki jih dodamo neodvisni mnozici. P je permutacija naravnih stevil do vkljucno len(V). Vozlisce dodamo I, ce ima najmanjso vrednost med sosedi.,tako tudi preprecimo ponavljanje vozlisc v mnozici.
        A = []      
        A.append(P[i])
        if list(G.neighbors(i)) != []:
            for j in list(G.neighbors(i)):
                A.append(P[j])
        if P[i] == min(A):
            I.append(i)
    N = [] 
    for i in I:
        N += G.neighbors(i)
    V1 = I + N
    V1 = list(dict.fromkeys(V1)) # V1 je mnozica, ki jo bomo odstranili iz grafa in je sestavljena iz vozlisc iz I, ter njihovih sosedov
    if I == []:
        return []
    else:
        G1 = G.copy()
        G1.remove_nodes_from(V1)
        return list(I + nakljucni_MIS(G1, P))

In [3]:
def tightness(graf, mnozica, v): #Dolocimo "tightness" vozlisca v, v mnozici X, tj. za vsakega soseda vozlisca v, v mnozici graf\X, tightness(v) povecamo za 1
    t = 0
    for w in graf.neighbors(v):
        if w in mnozica:
            t += 1
        else:
            pass
    return t

def lokalno_iskanje(G, I): #Parametra graf v networkx formatu, neodvisna množica I (gre za Python list)
    V = list(G.nodes)
    for v in I:
        L = [] #Gradimo mnozice potencialnih kandidatov za dodajanje v neodvisno mnozico
        for w in G.neighbors(v):
            if G.neighbors(v) != []:
                if tightness(G, I, w) == 1:
                    L.append(w)
                else:
                    pass
            else:
                pass
        if len(L) >= 2:
            for v1 in L:
                for w1 in L:
                    if w1 not in G.neighbors(v1):
                        I.remove(v)
                        I_lokalno = I + [v1, w1]
                        return lokalno_iskanje(G, I_lokalno)
        else:
            return I 

In [4]:
def CLP(G):
    p = MixedIntegerLinearProgram(maximization=True)
    x = p.new_variable(binary=True)
    p.set_objective(sum(x[v] for v in G))

    for u, v in G.edges(labels=False):
        p.add_constraint(x[u] + x[v] <= 1)
    
    p.solve()
    x = p.get_values(x)
    return [v for v, i in x.items() if i]

In [ ]:
#Ista vozlisca in verjetnost, veckrat

In [97]:
A = []
for i in range(1000):
    G = nx.erdos_renyi_graph(50, 0.3)
    G_slovar = nx.to_dict_of_lists(G)
    A += [G_slovar]

In [102]:
np1000_nakljucno = []
casi_nakljucno = []
resitve = []
for i in A:
    time1 = time()
    nak = nakljucni_MIS(nx.Graph(i), list(np.random.permutation(len(nx.Graph(i).nodes))))
    time2 = time()
    np1000_nakljucno.append(len(nak))
    casi_nakljucno.append(time2 - time1)
    resitve.append(nak)

In [100]:
np1000_CLP = []
casi_CLP = []
for i in A:
    X = Graph()
    from_dict_of_lists(X, i)
    time1 = time()
    rez = CLP(X)
    time2 = time()
    np1000_CLP.append(len(rez))
    casi_CLP.append(time2 - time1)

In [103]:
np1000_lokalno = []
casi_lokalno = []
for i in range(len(A)):
    time1 = time()
    rez = lokalno_iskanje(nx.Graph(A[i]), resitve[i])
    time2 = time()
    np1000_lokalno.append(len(rez))
    casi_lokalno.append(time2 - time1)

In [109]:
data_np1000 = {"Nakljucno":np1000_nakljucno, "Casi nakljucno": casi_nakljucno,
           "CLP": np1000_CLP, "Casi CLP": casi_CLP ,
            "Lokalno iskanje": np1000_lokalno, "Casi lokalno": casi_lokalno}
data_np1000 = pd.DataFrame(data_np)
print(data_np1000)



     Nakljucno  Casi nakljucno  CLP  Casi CLP  Lokalno iskanje  Casi lokalno
0            9        0.003369   11  0.156384                9      0.001147
1            8        0.003180   11  0.134443                9      0.000873
2            7        0.003683   11  0.128538                7      0.001180
3            9        0.003485   11  0.128925                9      0.000886
4            7        0.003106   10  0.192122                9      0.000958
..         ...             ...  ...       ...              ...           ...
995          9        0.002117   11  0.153980               10      0.001072
996         10        0.002280   11  0.164795               10      0.000790
997         10        0.002250   11  0.168162               10      0.000735
998          9        0.002342   13  0.097588                9      0.000670
999          9        0.002402   10  0.200561                9      0.000680

[1000 rows x 6 columns]


In [107]:
#Razlicna vozlisca, ista verjetnost



In [86]:
#Ista vozlisca, razlicna verjetnost

